In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Library to process text
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# Load the English language model
!pip download spacy-transformers
!python -m spacy download en_core_web_trf

import spacy_transformers
nlp = spacy.load("en_core_web_trf")

import re

# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer(language='english')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.0/493.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.6/920.6 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
df = pd.read_csv('/kaggle/input/imdb-featured-film/cleaned_data/cleaned_movie_data.csv')
df

,title,img_url,year,director,stars,duration,genres,overview,rating,num_rating,...,Adventure,Crime,Music,estimated_budget,budget_usd,estimated_gross,gross_usd,win,nomination,movie_id
0,The Kid,https://m.media-amazon.com/images/M/MV5BZjhhMT...,1921,Charles Chaplin,"Charles Chaplin,Edna Purviance,Jackie Coogan,C...",68,"['Comedy', 'Drama', 'Family']","The Tramp cares for an abandoned child, but ev...",8.2,132000,...,0,0,0,250000.0,2.500000e+05,41960.0,41960.0,2,0,tt0012349
1,Sherlock Jr.,https://m.media-amazon.com/images/M/MV5BZWFhOG...,1924,Buster Keaton,"Buster Keaton,Kathryn McGuire,Joe Keaton,Buste...",45,"['Action', 'Comedy', 'Romance']","A film projectionist longs to be a detective, ...",8.2,54000,...,0,0,0,NaN,NaN,NaN,NaN,2,0,tt0015324
2,The Gold Rush,https://m.media-amazon.com/images/M/MV5BZjEyOT...,1925,Charles Chaplin,"Charles Chaplin,Mack Swain,Tom Murray,Charles ...",95,"['Adventure', 'Comedy', 'Drama']",A prospector goes to the Klondike during the 1...,8.1,116000,...,1,0,0,923000.0,9.230000e+05,29328.0,29328.0,3,3,tt0015864
3,The General,https://m.media-amazon.com/images/M/MV5BYmRiMD...,1926,Clyde Bruckman,"Buster Keaton,Marion Mack,Glen Cavender,Buster...",78,"['Action', 'Adventure', 'Comedy']",After being rejected by the Confederate milita...,8.1,96000,...,1,0,0,750000.0,7.500000e+05,NaN,NaN,2,1,tt0017925
4,Metropolis,https://m.media-amazon.com/images/M/MV5BMTg5YW...,1927,Fritz Lang,"Brigitte Helm,Alfred Abel,Gustav Fröhlich,Brig...",153,"['Drama', 'Sci-Fi']",In a futuristic city sharply divided between t...,8.3,181000,...,0,0,0,6000000.0,9.836066e+06,1349711.0,1349711.0,6,6,tt0017136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58623,Ryadom,https://m.media-amazon.com/images/M/MV5BNGZiOW...,2022,Tamara Dondurey,"Aleksandr Alyabev,Ingeborga Dapkunaite,Tatyana...",80,['Drama'],Kira is a 30-year-old successful architect liv...,5.6,37,...,0,0,0,25000000.0,1.923077e+09,8691.0,8691.0,0,1,tt16995280
58624,3E,https://m.media-amazon.com/images/M/MV5BNGMwZj...,2022,Vishwanath Batchu,"Sagar Kiran,Sagar Kiran,NikhilThatiparthy,Saga...",110,"['Crime', 'Drama', 'Thriller']",Following the closing arguments in a murder tr...,8.5,1200,...,0,1,0,NaN,NaN,NaN,NaN,0,0,tt17514144
58625,1000 deshevykh zazhigalok,https://m.media-amazon.com/images/M/MV5BYTc1N2...,2022,Irina Obidova,"Pavel Chernyshyov,Elizaveta Kononova,Nikita Pl...",91,['Drama'],Seventeen-year-old Artyom is invisible in the ...,5.6,33,...,0,0,0,NaN,NaN,18465.0,18465.0,0,1,tt23723526
58626,Spider-Man: Across the Spider-Verse,https://m.media-amazon.com/images/M/MV5BMzI0Nm...,2023,Joaquim Dos Santos,"Shameik Moore,Hailee Steinfeld,Brian Tyree Hen...",140,"['Animation', 'Action', 'Adventure']","Miles Morales catapults across the Multiverse,...",8.7,278000,...,1,0,0,150000000.0,1.500000e+08,690001944.0,690001944.0,4,1,tt9362722


In [3]:
def process_text(text):
    
    # Uncapitalizing -> same format
    text = text.lower()
    # Process the text with spacy
    doc = nlp(text)
    # Reserve entity name
    entities = doc.ents

    filtered_tokens = []
    
    for token in doc:
        skip = False
        for entity in entities:
            if token.text in entity.text:
                skip = True
                break
        if skip:
            filtered_tokens.append(token.text)
        else:
            if not token.is_stop:
                filtered_tokens.append(re.sub(r'[^\w\s]', '', token.lemma_))

    # Join the lemmatized tokens back into a sentence
    filtered_sentence = ' '.join(filtered_tokens)    
    return filtered_sentence

In [4]:
df['processed_overview'] = df['overview'].apply(process_text)

In [5]:
df['processed_title'] = df['title'].apply(process_text)

In [6]:
df.to_csv('processed_movie_data.csv')